In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import pickle
import numpy as np

In [3]:
from openslide import OpenSlide

In [4]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi

from exact_sync.v1.models import ImageSet, Product, AnnotationType, Image, Annotation
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

In [29]:
# Python < 3.8?
import sys
#! pip install pickle5

if sys.version_info.minor < 8:
    import pickle5 as pickle

In [5]:
import sys
sys.path.append("../..")
from registration_tree import Rect, QuadTree

In [6]:
slide_list = {path.name: path for path in Path(r"D:\Datasets\ColonCancer\SingleSlide").glob("*/*.*") 
                      if path.suffix in [".tif"]}

slide_list

{'CRC-AI-11 F X40.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/HE/CRC-AI-11 F X40.tif'),
 'CRC-AI-14 1G X40.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/HE/CRC-AI-14 1G X40.tif'),
 'CRC-AI-15 X40.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/HE/CRC-AI-15 X40.tif'),
 'CRC-AI-8 E X40.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/HE/CRC-AI-8 E X40.tif'),
 'CRC-AI-8 H X40.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/HE/CRC-AI-8 H X40.tif'),
 'CRC-AI-11 F X40 IHC.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/IHC/CRC-AI-11 F X40 IHC.tif'),
 'CRC-AI-14 1G X40 IHC.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/IHC/CRC-AI-14 1G X40 IHC.tif'),
 'CRC-AI-15 X40 IHC.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/IHC/CRC-AI-15 X40 IHC.tif'),
 'CRC-AI-8 E X40 IHC.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/IHC/CRC-AI-8 E X40 IHC.tif'),
 'CRC-AI-8 H X40 IHC.tif': WindowsPath('D:/Datasets/ColonCancer/SingleSlide/IHC/CRC

In [7]:
configuration = Configuration()
configuration.username = 'exact' #'exact'
configuration.password = 'exact' #'exact'
configuration.host = "http://127.0.0.1:1337" #"http://127.0.0.1:8000"

client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

In [10]:
annotation_types_points = {}

for product in product_api.list_products(name="Reference_Points").results:
    for annotation_type in annotation_types_api.list_annotation_types(product=product.id).results:
        annotation_types_points[annotation_type.id] = annotation_type

annotation_types_points

{21: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 21,
  'name': 'L0',
  'node_count': 0,
  'product': 7,
  'sort_order': 0,
  'vector_type': 1},
 22: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 22,
  'name': 'L1',
  'node_count': 0,
  'product': 7,
  'sort_order': 0,
  'vector_type': 1},
 23: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 23,
  'name': 'L2',
  'node_count': 0,
  'product': 7,
  'sort_order': 0,
  'vector_type': 1},
 24: {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 100,
  'default_width': 100,
  'enable_blurred': False,
  'e

In [15]:
annotations = []

image_set = image_sets_api.list_image_sets(name="Single_IHC", expand="product_set,product_set.annotationtype_set").results[0]

for image_id in image_set.images:
    
    image = images_api.retrieve_image(id=image_id)
    
    for anno in annotations_api.list_annotations(image=image_id,
                                                 deleted=False, fields="annotation_type,id,image,vector,unique_identifier").results:
        
        if anno.annotation_type in annotation_types_points:
            short = Path(image.name).stem.split("X40")[0]
            
            annotations.append([image.id, image.name, short, anno.id, anno.vector, anno.unique_identifier, anno.annotation_type])
        
annotations = pd.DataFrame(annotations, columns=["image_id", "image_name", "image_name_short", "id", "vector", "unique_identifier", "annotation_type"])
annotations

,image_id,image_name,image_name_short,id,vector,unique_identifier,annotation_type
0,324,CRC-AI-15 X40 IHC.tif,CRC-AI-15,25981,"{'x1': 7227, 'x2': 7427, 'y1': 9959, 'y2': 10159}",f52b1e31-3df8-4041-9902-66bc6edef761,21
1,324,CRC-AI-15 X40 IHC.tif,CRC-AI-15,25982,"{'x1': 7227, 'x2': 7427, 'y1': 24829, 'y2': 25...",b8ca1cea-38ef-4818-b050-d77ef0e44e21,22
2,324,CRC-AI-15 X40 IHC.tif,CRC-AI-15,25983,"{'x1': 7227, 'x2': 7427, 'y1': 39699, 'y2': 39...",9d817eda-dd12-49b4-a942-9d7ddd77f305,23
3,324,CRC-AI-15 X40 IHC.tif,CRC-AI-15,25984,"{'x1': 7227, 'x2': 7427, 'y1': 54569, 'y2': 54...",04454db4-2e29-4e18-8f8e-271d8e154507,24
4,324,CRC-AI-15 X40 IHC.tif,CRC-AI-15,25985,"{'x1': 7227, 'x2': 7427, 'y1': 69439, 'y2': 69...",81ee0679-7857-4fc4-a402-df65b0576831,25
...,...,...,...,...,...,...,...
120,331,CRC-AI-14 1G X40 IHC.tif,CRC-AI-14 1G,26101,"{'x1': 58642, 'x2': 58842, 'y1': 9417, 'y2': 9...",c087f152-ee2c-4d39-b350-5f0a19278d3f,41
121,331,CRC-AI-14 1G X40 IHC.tif,CRC-AI-14 1G,26102,"{'x1': 58642, 'x2': 58842, 'y1': 26993, 'y2': ...",0bd2dfcc-79da-439b-9200-b405ebea8406,42
122,331,CRC-AI-14 1G X40 IHC.tif,CRC-AI-14 1G,26103,"{'x1': 58642, 'x2': 58842, 'y1': 44569, 'y2': ...",a4bec180-cd41-4c78-b899-32d8f43487ad,43
123,331,CRC-AI-14 1G X40 IHC.tif,CRC-AI-14 1G,26104,"{'x1': 58642, 'x2': 58842, 'y1': 62145, 'y2': ...",e12a6e56-a5c4-4151-9581-a6217ef91aaa,44


In [17]:
annotations["x1"] = [vector['x1'] if type(vector) is dict else 0 for vector in annotations["vector"]]
annotations["y1"] = [vector['y1'] if type(vector) is dict else 0 for vector in annotations["vector"]]

annotations["x2"] = [vector['x2'] if type(vector) is dict else 0 for vector in annotations["vector"]]
annotations["y2"] = [vector['y2'] if type(vector) is dict else 0 for vector in annotations["vector"]]

annotations["center_x"] = [x1 + ((x2-x1) / 2) for x1, x2 in zip(annotations["x1"], annotations["x2"])]
annotations["center_y"] = [y1 + ((y2-y1) / 2) for y1, y2 in zip(annotations["y1"], annotations["y2"])]

annotations["anno_width"] = [x2-x1 for x1, x2 in zip(annotations["x1"], annotations["x2"])]
annotations["anno_height"]= [y2-y1 for y1, y2 in zip(annotations["y1"], annotations["y2"])]

In [35]:
transformed_annotations_to_upload = []

for image_set_name in tqdm(["Single_HE"]):
    image_set = image_sets_api.list_image_sets(name=image_set_name, expand="product_set,product_set.annotationtype_set").results[0]

    for image_id in image_set.images:
        
        target_image = images_api.retrieve_image(id=image_id)
        short = Path(target_image.name).stem.replace("40X", "X40").split("X40")[0]
        
        source_annos = annotations[annotations["image_name_short"] == short]
        
        source_name = list(source_annos["image_name"].unique())[0]
        target_name = target_image.name
        
        qtree = pickle.load(open(f'../../ColonCancer/Depth_2/2048/{Path(source_name).stem}-To-{Path(target_name).stem}.pickle', "rb" ))
        #qtree.source_slide = OpenSlide(str(slide_list[source_name]))
        #qtree.target_slide = OpenSlide(str(slide_list[target_name]))
        
        for id, source_anno in source_annos.iterrows():

            box = [source_anno.center_x, source_anno.center_y, source_anno.anno_width, source_anno.anno_height]

            trans_box = qtree.transform_boxes(np.array([box]))[0]

            new_x1 = int(trans_box[0] - trans_box[2] // 2)
            new_y1 = int(trans_box[1] - trans_box[3] // 2)
            new_x2 = int(trans_box[0] + trans_box[2] // 2)
            new_y2 = int(trans_box[1] + trans_box[3] // 2)

            vector = {'x1': new_x1, 'x2': new_x2, 'y1': new_y1, 'y2': new_y2}

            anno = Annotation(annotation_type=source_anno.annotation_type, 
                                  vector=vector, 
                                  image=target_image.id,
                                 unique_identifier=source_anno.unique_identifier)

            transformed_annotations_to_upload.append(anno)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.17s/it]


In [36]:
transformed_annotations_to_upload[:3]

[{'annotation_type': 21,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 325,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': 'f52b1e31-3df8-4041-9902-66bc6edef761',
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 6329, 'x2': 6527, 'y1': 8953, 'y2': 9151},
  'verified_by_user': None},
 {'annotation_type': 22,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 325,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': 'b8ca1cea-38ef-4818-b050-d77ef0e44e21',
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 6448, 'x2': 6648, 'y1': 23852, 'y2': 24052},
  'verified_by_user': None},
 {'annotation_type': 23,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': 

In [37]:
annotations_api.create_annotation(body=transformed_annotations_to_upload)

[{'annotation_type': 21,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 26106,
  'image': 325,
  'last_edit_time': datetime.datetime(2021, 2, 9, 13, 55, 43, 262489),
  'last_editor': 2,
  'meta_data': None,
  'time': datetime.datetime(2021, 2, 9, 13, 55, 43, 262456),
  'unique_identifier': 'f52b1e31-3df8-4041-9902-66bc6edef761',
  'uploaded_media_files': [],
  'user': 2,
  'vector': {'x1': 6329, 'x2': 6527, 'y1': 8953, 'y2': 9151},
  'verified_by_user': 'False'},
 {'annotation_type': 22,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 26107,
  'image': 325,
  'last_edit_time': datetime.datetime(2021, 2, 9, 13, 55, 43, 262570),
  'last_editor': 2,
  'meta_data': None,
  'time': datetime.datetime(2021, 2, 9, 13, 55, 43, 262555),
  'unique_identifier': 'b8ca1cea-38ef-4818-b050-d77ef0e44e21',
  'uploaded_media_files': [],
  'user': 2,
